In [2]:
# created on Dec 24, 2020
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

!python -m pip install tweepy
import tweepy, json, time

In [6]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.f = open(file, 'a', encoding="utf-8")
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                # {'limit': {'track': 13, 'timestamp_ms': '1585851016736'}}

                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    record = '%s, %s, %f, %f, %s \n' % (username, created_at, lng, lat, text)
                    print(record)
                    self.f.write(record)
                else:
                    pass
        else:
            self.f.close()
            print("finished")
            return False

In [7]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = "assets/tweets.csv"

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "Z4ArnpojuQmr0gBoIS69Ts5HY"
    consumer_secret = "6iUY8ZkZ8a5vE5ovWQe0JZGiHqPpLTkDLKZ91R4I2JhjGWlX39"
    access_token = "1222649151814430720-6L4EKozDfNdhpAtklEvGTf9zegS6Yf"
    access_token_secret = "Fu0iE8o6iZ5NrAlWVYelEQDg5JERcyfAIMWGKbtp2Z2Gq"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-124.7771694, 24.520833, -66.947028, 49.384472,  # Contiguous US
                 -164.639405, 58.806859, -144.152365, 71.76871,  # Alaska
                 -160.161542, 18.776344, -154.641396, 22.878623]  # Hawaii

    stream_listener = StreamListener(time_limit=60, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS)
    stream.filter(track=['protests'], is_async=True)
    

{'created_at': 'Wed Apr 22 14:37:03 +0000 2020', 'id': 1252969673232007169, 'id_str': '1252969673232007169', 'text': '@BarbIllinois10 @HKrassenstein I want Kamala for Attorney General', 'display_text_range': [31, 65], 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': 1252962618618281984, 'in_reply_to_status_id_str': '1252962618618281984', 'in_reply_to_user_id': 803009244924624896, 'in_reply_to_user_id_str': '803009244924624896', 'in_reply_to_screen_name': 'BarbIllinois10', 'user': {'id': 814204489, 'id_str': '814204489', 'name': '🌊🌊💙💙EmilyLoves', 'screen_name': 'NotaryEM', 'location': 'Kent, WA', 'url': 'https://confessionsoftherelocatedspouse.wordpress.com/', 'description': 'Bariatric Gastric sleeve, excellent Home Cook, Blogger, Wife and Puppy Mom to spoiled Schnauzers.', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 485, 'friends_count': 1121, 'listed_c

{'created_at': 'Wed Apr 22 14:37:04 +0000 2020', 'id': 1252969674272239618, 'id_str': '1252969674272239618', 'text': '#SunriseGratitude', 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 62844701, 'id_str': '62844701', 'name': 'Marisa 💫', 'screen_name': 'hashtagmarisa', 'location': 'San Diego, CA', 'url': None, 'description': 'Personal tweets sometimes including things I love: futbol, tech, space 📸,  & the Arsenal... Black coffee is life | Co-host🎙️ @fair_pod ❤️⚽ #WomenInSports', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 2603, 'friends_count': 2443, 'listed_count': 137, 'favourites_count': 6362, 'statuses_count': 88455, 'created_at': 'Tue Aug 04 15:24:43 +0000 2009', 'utc_offset': None, 'time_zone': Non

{'created_at': 'Wed Apr 22 14:37:04 +0000 2020', 'id': 1252969675136344064, 'id_str': '1252969675136344064', 'text': 'me losing respect for you is worse than me being mad', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1143211379635216384, 'id_str': '1143211379635216384', 'name': 'ig @theofficialkai2x 🧡', 'screen_name': 'kaiposted', 'location': 'RatchetCity Fort Worth', 'url': None, 'description': 'Aren’t you prada me ? 🕺🏽 raise hell, not kids 🤣 18 📍@toosii2x my boyfriennnnnn 💍 FOLLOW ME 😛', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 2262, 'friends_count': 2268, 'listed_count': 3, 'favourites_count': 11126, 'statuses_count': 13558, 'created_at': 'Mon Jun 24 17:36:47 +0000 2019', 'utc_offset': None, 'tim

{'created_at': 'Wed Apr 22 14:37:04 +0000 2020', 'id': 1252969676180725760, 'id_str': '1252969676180725760', 'text': 'the video for this song has me actually crying, the ghost inside are amazing and i’m so so proud of them \U0001f90d', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 387661063, 'id_str': '387661063', 'name': 'cam', 'screen_name': 'HexvenSent', 'location': 'north carolina // 19', 'url': 'http://onlyfans.com/fivestarslut', 'description': 'ᏣᎳᎩ & mescalero apache • aquarius • the girl at the rock show • 18+ account: @fivestarslut', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 419, 'friends_count': 392, 'listed_count': 2, 'favourites_count': 10398, 'statuses_count': 10491, 'created_at': 'Sun Oct 

{'created_at': 'Wed Apr 22 14:37:04 +0000 2020', 'id': 1252969677090951171, 'id_str': '1252969677090951171', 'text': '@i_am_tahj chubbs upset ain’t it lls', 'display_text_range': [11, 36], 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': 1252727105944772608, 'in_reply_to_status_id_str': '1252727105944772608', 'in_reply_to_user_id': 1177611607, 'in_reply_to_user_id_str': '1177611607', 'in_reply_to_screen_name': 'i_am_tahj', 'user': {'id': 1196660615567347712, 'id_str': '1196660615567347712', 'name': '$leazz.', 'screen_name': 'biqsleazz', 'location': 'Coles Point, VA ', 'url': None, 'description': 'shid is nice... but i prefer Madéo. rip @bigsleazz', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 145, 'friends_count': 99, 'listed_count': 0, 'favourites_count': 5274, 'statuses_count': 12371, 'created_at': 'Tue Nov 19 05:25:11 +0000 2019', 'utc_offset': None, 't

...

{'created_at': 'Wed Apr 22 14:38:02 +0000 2020', 'id': 1252969919316217859, 'id_str': '1252969919316217859', 'text': '@DwnunderDeciple @ClemonsJake Nah, never heard of him.', 'display_text_range': [30, 54], 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': 1252967656640004096, 'in_reply_to_status_id_str': '1252967656640004096', 'in_reply_to_user_id': 1240148395827888128, 'in_reply_to_user_id_str': '1240148395827888128', 'in_reply_to_screen_name': 'DwnunderDeciple', 'user': {'id': 2889917630, 'id_str': '2889917630', 'name': 'D.A. Brewer', 'screen_name': 'DABrewerWWE', 'location': 'Orlando, FL', 'url': 'http://www.WWE.com', 'description': "Trained at #WWA4. I've been up and down the Highways in Tights and Zebra Stripes. Now I count 1-2-3 for @WWENXT #NXT #WeAreNXT  #ZebraArmy #HardBodyRefs", 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 6673, 'friends_count': 

{'created_at': 'Wed Apr 22 14:38:02 +0000 2020', 'id': 1252969920310251520, 'id_str': '1252969920310251520', 'text': 'Who has the BEST counselor in @TeamHISD? @WeRFoerster are blessed to have @CounselorShealy who keeps a watch over o… https://t.co/aiJThTVQU6', 'display_text_range': [0, 140], 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': True, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1368662346, 'id_str': '1368662346', 'name': 'Gina Randall-Wilkins', 'screen_name': 'grandall47', 'location': 'Houston, TX', 'url': None, 'description': 'Child of God, Wife, Mother, AKA, and Educator', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 190, 'friends_count': 271, 'listed_count': 0, 'favourites_count': 2215, 'statuses_count': 718, 'created_at': 'Sun Apr 21 04:45:1

{'created_at': 'Wed Apr 22 14:38:02 +0000 2020', 'id': 1252969921069363203, 'id_str': '1252969921069363203', 'text': '@YoursTrulyDes__ https://t.co/d8gUx5i3wx', 'display_text_range': [16, 16], 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': 1252968813051723776, 'in_reply_to_status_id_str': '1252968813051723776', 'in_reply_to_user_id': 350333194, 'in_reply_to_user_id_str': '350333194', 'in_reply_to_screen_name': 'YoursTrulyDes__', 'user': {'id': 345725380, 'id_str': '345725380', 'name': 'B.WELL', 'screen_name': 'B_Well_', 'location': 'The Hills • 515', 'url': 'https://bwell.biglink.to/atpeace', 'description': 'Recording Artist🎙 • @DeadStockEnt✖️• Philanthropist🌱• Event Coordinator 🎫 • Creative Director 🎨', 'translator_type': 'regular', 'protected': False, 'verified': False, 'followers_count': 1186, 'friends_count': 907, 'listed_count': 7, 'favourites_count': 26688, 'statuses_count': 17704, 'crea

{'created_at': 'Wed Apr 22 14:38:03 +0000 2020', 'id': 1252969923149774850, 'id_str': '1252969923149774850', 'text': 'Just posted a photo @ Las Vegas, Nevada https://t.co/lvq35AAu24', 'source': '<a href="http://instagram.com" rel="nofollow">Instagram</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1274139060, 'id_str': '1274139060', 'name': 'shaun singleton', 'screen_name': 'singleton_shaun', 'location': 'LAS VEGAS NV', 'url': None, 'description': 'I DO ALL STRIP CLUBS IN LAS VEGAS NV I HAVE OVER 20 YEARS EXPERIENCE IN THE INDUSTRY\nFREE LIMOUSINE RIDE ROUND TRIP\nFREE COVER\nFREE VIP TABLE\nCALL (702)931-6070', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 444, 'friends_count': 4848, 'listed_count': 2, 'favourites_count': 12292, 'statuses_count': 8106, 'created_at': 'Sun Mar 17 05:52:13 +0000 20

finished
